In [ ]:
from pathlib import Path
from pprint import pprint

import pandas as pd

import graphrag.api as api
from graphrag.config.load_config import load_config
from graphrag.index.typing.pipeline_run_result import PipelineRunResult

### handson ディレクトリ概要
- `.env`: Graphrag 実行時の環境変数を配置する。
- `settings.yaml`: Graphrag の設定ファイル。本演習のパラメータが定義されている。
- `input/`: インデックス化対象となる元ドキュメントを格納する。
- `output/`: build_index 実行後に作成されるエンティティやリレーションなどの成果物が保存される。
- `cache/`・`logs/`: 実行時のキャッシュやログを保持し、再実行時の高速化やトラブルシュートに使う。
- `prompts/`: 追加でカスタムプロンプトを定義したい場合に配置する。

このセルでは `handson` ディレクトリにある `settings.yaml` から設定を読み込み、`graphrag.api.build_index` を実行してインデックスを構築します。各ワークフローの成功／失敗が標準出力に表示されるため、実行後にログを確認して次の処理（データの可視化や検索）へ進みます。

In [ ]:
PROJECT_DIRECTORY = "./handson"
graphrag_config = load_config(Path(PROJECT_DIRECTORY))

In [ ]:
index_result: list[PipelineRunResult] = await api.build_index(config=graphrag_config)

for workflow_result in index_result:
    status = f"error\n{workflow_result.errors}" if workflow_result.errors else "success"
    print(f"Workflow Name: {workflow_result.workflow}\tStatus: {status}")

In [ ]:
entities_df = pd.read_parquet(f"{PROJECT_DIRECTORY}/output/entities.parquet")
communities_df = pd.read_parquet(f"{PROJECT_DIRECTORY}/output/communities.parquet")
community_reports_df = pd.read_parquet(
    f"{PROJECT_DIRECTORY}/output/community_reports.parquet"
)
relationships_df = pd.read_parquet(f"{PROJECT_DIRECTORY}/output/relationships.parquet")
documents_df = pd.read_parquet(f"{PROJECT_DIRECTORY}/output/documents.parquet")
text_units_df = pd.read_parquet(f"{PROJECT_DIRECTORY}/output/text_units.parquet")

#### Entity カラム概要
- `id`: 解析パイプライン内でユニークなエンティティ ID（UUID）。
- `human_readable_id`: 連番の簡易 ID。グラフ表示やデバッグに便利。
- `title`: エンティティ名。後続の可視化やクエリでノードラベルとして利用。
- `type`: LLM が推定したカテゴリ（PERSON/ORGANIZATION など）。
- `description`: エンティティの説明文。検索応答の根拠として活用。
- `text_unit_ids`: このエンティティが登場するテキストユニット ID のリスト。
- `frequency`: エンティティが参照された回数。重要度の簡易スコアとして使える。
- `degree`: グラフ上で接続している関係数。ハブノードの特定に有用。
- `x`, `y`: レイアウト計算済みの座標。ノートブック内でグラフを描画する際に使用。

In [ ]:
entities_df

#### Relationships カラム概要
- `id`: 関係エッジのユニーク ID（UUID）。
- `human_readable_id`: 連番で付与された簡易 ID。
- `source` / `target`: エッジが結ぶエンティティのタイトル。`entities_df['title']` と対応。
- `description`: LLM が生成した関係の説明文。
- `weight`: 関係の重要度スコア。GraphWidget の線幅などに利用。
- `combined_degree`: 接続するノード双方の次数から計算された指標。ハブ間の関係を把握しやすい。
- `text_unit_ids`: 関係が出現したテキストユニット ID の一覧。根拠ドキュメント追跡に使える。

In [ ]:
relationships_df

### Community Reports カラム概要
- `id`: コミュニティレポートのユニーク ID（UUID）。
- `human_readable_id`: 連番で付いた簡易 ID。
- `community`: レポート対象コミュニティの ID。`communities_df['id']` と対応。
- `level`: コミュニティの階層レベル。0（グローバル）～ N。
- `parent` / `children`: 階層構造を表す親コミュニティ ID と子コミュニティ ID のリスト。
- `title`: レポートの見出しとして使われる要約タイトル。
- `summary`: コミュニティ全体の要約文。
- `full_content`: 箇条書きやセクションを含む詳細レポート本文。
- `full_content_json`: `full_content` を JSON 構造化したバージョン。後続の加工や UI 連携で利用。
- `findings`: 主要な洞察・リスク・推奨事項を配列で保持。
- `rank`: コミュニティの重要度スコア。
- `rating_explanation`: ランク付けに関する説明文。
- `period`: 解析対象期間など、レポートに紐づく時間情報（存在すれば）。
- `size`: コミュニティに属するテキストユニット数。

In [ ]:
community_reports_df

## Local Search の仕組み

1. **ユーザーのクエリ**（＋過去の会話履歴）が入力される。  
2. クエリ内容に基づき、**知識グラフから意味的に関連する可能性のあるエンティティ**を抽出する（embedding 類似度などを利用）。  
3. 抽出したエンティティを起点に、以下の候補情報を収集する：  
   - 元ドキュメントのテキストチャンク（Entity-Text Units）  
   - 関関連エンティティや関係性（Entity-Entity / Relationships）  
   - エンティティ属性・メタデータ（Covariates）  
   - コミュニティ構造・レポート（Community Reports）  
4. 収集したチャンク・エンティティ・関係・レポートを**ランキングとフィルタリング**で精査し、使用する情報のみを選定する。  
5. 選ばれた情報（テキスト + 構造化データ）を **LLM に入力し、最終的な回答を生成**する。

> Local Search は、全文検索だけでなく **知識グラフの構造化情報とテキスト情報を組み合わせるハイブリッド手法**で、高精度なエンティティ中心検索を実現します。

参考：https://microsoft.github.io/graphrag/query/local_search/


In [ ]:
response, context = await api.local_search(
    config=graphrag_config,
    entities=entities_df,
    communities=communities_df,
    community_reports=community_reports_df,
    text_units=text_units_df,
    relationships=relationships_df,
    covariates=None,
    community_level=2,
    response_type="Multiple Paragraphs",
    query="Azure OpenAIで使えるモデルを教えて",
)

In [ ]:
print(response)

## Global Search の仕組み

1. ドキュメント群からナレッジグラフを構築し、グラフ内のエンティティが属する **コミュニティ (Community)** を検出／クラスタリングする。  
2. 各コミュニティに対して、LLM による要約を作成し **Community Report（コミュニティ要約）** を生成する。  
3. ユーザーのクエリに対し、指定レベルのコミュニティ階層に属するすべての Community Report を対象に、Map-Reduce 方式で応答生成を試みる。  
   - Map工程：Community Report を一定サイズのテキストチャンクに分割 → 各チャンクごとに中間応答 (Intermediate Response) を生成  
   - Reduce工程：すべての中間応答をフィルタ／統合 → 最終的な応答を生成  
4. 必要に応じて、コミュニティ階層のレベル (抽象度) を調整 — 抽象度の低いレポートは情報量が多く、詳細な応答が可能だが、コストやトークン消費が増える。  
5. この手法により、ドキュメント全体を俯瞰した **広範なテーマ、全体構造、総合的傾向** への問いに対して、一貫性のある総合的な回答を返すことが可能となる。  

参考：https://microsoft.github.io/graphrag/query/global_search/

In [ ]:
response, context = await api.global_search(
    config=graphrag_config,
    entities=entities_df,
    communities=communities_df,
    community_reports=community_reports_df,
    community_level=2,
    dynamic_community_selection=False,
    response_type="Multiple Paragraphs",
    query="Azure OpenAIで使えるモデルを教えて",
)

In [ ]:
print(response)

## DRIFT Search の仕組み

1. ユーザーのクエリが入力されると、まず **“コミュニティ要約 (Community Report)”** の集合の中から、クエリと意味的に最も関連性の高い上位 K 件を選び出し、**初期回答＋フォローアップ質問** を生成する。  
2. 生成されたフォローアップ質問に対しては、**Local Search の仕組み**を使って、より詳細な情報の探索と中間回答の収集を行う。  
3. フォローアップ→Local Search→さらに質問生成、というサイクルを、あらかじめ定められた条件 (繰り返し回数やスコア閾値) に基づいて繰り返すことで、情報の深掘りと精度向上を図る。  
4. 最終的に、Global 的視点（コミュニティ要約ベース）と Local 的視点（テキストチャンク＋エンティティ構造ベース）が統合された、**階層構造つきの質問／回答群** を生成する。  
5. このアプローチにより、広範囲な知識の「俯瞰 (global)」と、個別情報の「精査 (local)」を両立させ、**多様かつ文脈に即した回答** を得ることが可能になる。  

参考：https://microsoft.github.io/graphrag/query/drift_search/

In [ ]:
response, context = await api.drift_search(
    config=graphrag_config,
    entities=entities_df,
    communities=communities_df,
    community_reports=community_reports_df,
    text_units=text_units_df,
    relationships=relationships_df,
    community_level=2,
    response_type="Multiple Paragraphs",
    query="Azure OpenAIで使えるモデルを教えて",
)

In [ ]:
print(response)